##### Imports the necessary modules and sets the system path to locate them.

In [ ]:
import sys
import networkx
import astra_sim_sdk.astra_sim_sdk as astra_sim_kit
sys.path.append("../../utils")
from astra_sim import AstraSim, Collective, NetworkBackend
from infragraph.infragraph_service import InfraGraphService
from infragraph.blueprints.fabrics.clos_fat_tree_fabric import ClosFatTreeFabric
from infragraph.blueprints.devices.server import Server
from infragraph.blueprints.devices.generic_switch import Switch



##### Connects the client to the AstraSim gRPC server, initializes the AstraSim SDK, and creates a folder (tagged as specified) containing all configuration details, generated results, and logs.

In [ ]:
astra = AstraSim(server_endpoint = "172.17.0.2:8989", tag = "infragraph_clos_3tier_trial")

##### Creating Infragraph for 3 tier clos fabric

In [ ]:
server = Server()
switch = Switch(port_count=4)
clos_fat_tree = ClosFatTreeFabric(switch, server, 3,[])
astra.configuration.infragraph.infrastructure.deserialize(clos_fat_tree.serialize())
print(astra.configuration.infragraph.infrastructure)

##### Display Fabric

In [ ]:
service = InfraGraphService()
service.set_graph(clos_fat_tree)
g = service.get_networkx_graph()
print(networkx.write_network_text(g, vertical_chains=True))
total_npus = 16

##### Generates workload execution traces for each rank and configures the data size, which is mandatory for AstraSim workload configuration.

In [ ]:
astra.configuration.common_config.workload = astra.generate_collective(collective=Collective.ALLREDUCE, coll_size= 8 *1024*1024, npu_range=[0, total_npus])

##### Configure the system configurations

In [ ]:
astra.configuration.common_config.system.scheduling_policy = astra.configuration.common_config.system.LIFO
astra.configuration.common_config.system.endpoint_delay = 10
astra.configuration.common_config.system.active_chunks_per_dimension = 1
astra.configuration.common_config.system.all_gather_implementation = [astra.configuration.common_config.system.RING]
astra.configuration.common_config.system.all_to_all_implementation = [astra.configuration.common_config.system.DIRECT]
astra.configuration.common_config.system.all_reduce_implementation = [astra.configuration.common_config.system.ONERING]
astra.configuration.common_config.system.collective_optimization = astra.configuration.common_config.system.LOCALBWAWARE
astra.configuration.common_config.system.local_mem_bw = 1600

##### Configure the remote memory configuration

In [ ]:
astra.configuration.common_config.remote_memory.memory_type = astra.configuration.common_config.remote_memory.NO_MEMORY_EXPANSION
print(astra.configuration.common_config.remote_memory)

##### Configure the network backend choice and the topology choice for that backend


In [ ]:
astra.configuration.network_backend.choice = astra.configuration.network_backend.NS3
astra.configuration.network_backend.ns3.topology.choice = astra.configuration.network_backend.ns3.topology.INFRAGRAPH
astra.configuration.network_backend.ns3.network.packet_payload_size = int(8192)


##### Adding ns3 trace and logical dimension 

In [ ]:
astra.configuration.network_backend.ns3.logical_topology.logical_dimensions = [total_npus]
astra.configuration.network_backend.ns3.trace.trace_ids = []
for i in range(0, total_npus):
    astra.configuration.network_backend.ns3.trace.trace_ids.append(i)

##### Adding ASTRA-sim specific annotation

In [ ]:
host_device_spec = astra_sim_kit.AnnotationDeviceSpecifications()
host_device_spec.device_bandwidth_gbps = 100
host_device_spec.device_latency_ms = 0.05
host_device_spec.device_name = "server"
host_device_spec.device_type = "host"
astra.configuration.infragraph.annotations.device_specifications.append(host_device_spec)

switch_device_spec = astra_sim_kit.AnnotationDeviceSpecifications()
switch_device_spec.device_bandwidth_gbps = 100
switch_device_spec.device_latency_ms = 0.05
switch_device_spec.device_name = "switch"
switch_device_spec.device_type = "switch"
astra.configuration.infragraph.annotations.device_specifications.append(
    switch_device_spec
)

##### Configure ASTRA-sim cmd parameters

In [ ]:
astra.configuration.common_config.cmd_parameters.comm_scale = 1
astra.configuration.common_config.cmd_parameters.injection_scale = 1
astra.configuration.common_config.cmd_parameters.rendezvous_protocol = False

#### Start the simulation by providing the network backend name in uppercase letters.

In [ ]:
astra.run_simulation(NetworkBackend.NS3)

##### Download all the configurations as a zip

In [ ]:
astra.download_configuration()

##### Read output files

In [ ]:
import pandas as pd
import os
from common import FileFolderUtils
df = pd.read_csv(os.path.join(FileFolderUtils.get_instance().OUTPUT_DIR, "fct.csv"))
df.head()

In [ ]:
df = pd.read_csv(os.path.join(FileFolderUtils.get_instance().OUTPUT_DIR, "flow_stats.csv"))
df.head()
